In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Lista de los 50 estados de EEUU

In [3]:
url = 'https://datosmacro.expansion.com/paro/usa-estados'

Scrapeamos la tabla

In [4]:
r1 = requests.get(url)
soup = BeautifulSoup(r1.content, 'html.parser')
rows = soup.find('table', attrs={'id':'tb1'}).find('tbody').find_all('tr')

Solo queremos la columna de estados,la primera columna

In [5]:
lista_estados = list()
for r in rows:
        lista_estados.append(r.find_all('td')[0].get_text())

Para el url necesitamos el nombre del estado sin mayuscula.<br>
Pasamos todo a minuscula

In [6]:
l_estados =  list(map(lambda x: x.lower(), lista_estados))

Cada valor de la columna tiene al final del nombre del estado ' [+]'<br>
Se lo quitamos

In [7]:
lista = list()
for i in l_estados:
    i = i.replace(' [+]', '')
    i = i.replace(' ', '-')
    lista.append(i)


In [8]:
lista

['alaska',
 'alabama',
 'arkansas',
 'arizona',
 'california',
 'colorado',
 'connecticut',
 'delaware',
 'florida',
 'georgia',
 'hawái',
 'iowa',
 'idaho',
 'illinois',
 'indiana',
 'kansas',
 'kentucky',
 'luisiana',
 'massachusetts',
 'maryland',
 'maine',
 'míchigan',
 'minnesota',
 'misuri',
 'misisipi',
 'montana',
 'carolina-del-norte',
 'dakota-del-norte',
 'nebraska',
 'nuevo-hampshire',
 'nueva-jersey',
 'nuevo-méxico',
 'nevada',
 'nueva-york',
 'ohio',
 'oklahoma',
 'oregón',
 'pensilvania',
 'rhode-island',
 'carolina-del-sur',
 'dakota-del-sur',
 'tennessee',
 'texas',
 'utah',
 'virginia',
 'vermont',
 'washington',
 'wisconsin',
 'virginia-occidental',
 'wyoming']

Armamos una lista con el intervalo de años a usar

In [9]:
años = ['2010', '2011','2012', '2013', '2014', '2015', '2016', '2017','2018','2019']

In [10]:
def CargaWeb(url):
    fecha = list()
    activos = list()
    tasa_desempleo = list()
    empleados = list()
    desempleados = list()

    for año in años:
        r = requests.get(url + año) 
        soup = BeautifulSoup(r.content, 'html.parser')
        rows = soup.find('table', attrs={'id':'tb0'}).find('tbody').find_all('tr')
                    
        for row in rows:
            fecha.append(row.find_all('td')[0].get_text())
            activos.append(row.find_all('td')[1].get_text())
            tasa_desempleo.append(row.find_all('td')[2].get_text())
            empleados.append(row.find_all('td')[3].get_text())
            desempleados.append(row.find_all('td')[4].get_text())

    df_estado = pd.DataFrame({'Fecha': fecha, 'Activos': activos, 
                            'Tasa de desempleo': tasa_desempleo,
                            'Empleados': empleados,
                            'Desempleados': desempleados,
                            'Estado': np.nan})
    return df_estado


El url para todos los estados segun el año es:<br>
'https://datosmacro.expansion.com/paro/usa-estados/**estado**?anio=**año**' <br>
Lo que varia en el url es el estado y el año

In [11]:
url_parte1 = 'https://datosmacro.expansion.com/paro/usa-estados/'
url_parte2 = '?anio='

Creamos el DataFrame final junto con todos los estados

In [12]:
df = pd.DataFrame(columns=('Fecha','Activos','Tasa de desempleo','Empleados','Desempleados','Estado'))

In [13]:
for i,e in enumerate(lista):
    state = CargaWeb(url_parte1 + e + url_parte2)
    state['Estado'] = e.capitalize()
    df = pd.concat([df, state])

In [14]:
df

,Fecha,Activos,Tasa de desempleo,Empleados,Desempleados,Estado
0,Diciembre 2010,"363,1","8,0%","333,9","29,20",Alaska
1,Noviembre 2010,"362,8","8,1%","333,5","29,22",Alaska
2,Octubre 2010,"362,2","8,0%","333,2","29,07",Alaska
3,Septiembre 2010,"361,7","8,0%","332,8","28,84",Alaska
4,Agosto 2010,"361,3","7,9%","332,7","28,66",Alaska
...,...,...,...,...,...,...
115,Mayo 2019,"293,4","3,4%","283,5","9,97",Wyoming
116,Abril 2019,"292,9","3,3%","283,1","9,80",Wyoming
117,Marzo 2019,"292,7","3,4%","282,8","9,92",Wyoming
118,Febrero 2019,"292,7","3,5%","282,4","10,29",Wyoming


Nos quedamos solo con las columnas que necesitamos

In [15]:
df = df.drop(['Activos', 'Empleados','Desempleados'], axis=1)

Quitamos el % y cambiamos coma por punto

In [16]:
df['Tasa de desempleo'] = df['Tasa de desempleo'].str.replace('%','')

In [17]:
df['Tasa de desempleo'] = df['Tasa de desempleo'].str.replace(',', '.')

Dividimos el mes y el año

In [18]:
df[['Mes','Año']]=df.Fecha.str.split(' ',expand=True)

Quitamos la columna fecha

In [19]:
df = df.drop(['Fecha'], axis=1)

In [20]:
df.head()

,Tasa de desempleo,Estado,Mes,Año
0,8.0,Alaska,Diciembre,2010
1,8.1,Alaska,Noviembre,2010
2,8.0,Alaska,Octubre,2010
3,8.0,Alaska,Septiembre,2010
4,7.9,Alaska,Agosto,2010


In [21]:
df = df.astype({'Tasa de desempleo': 'float64', 'Año': 'int32'})

Reordenamos columnas

In [22]:
df = df.reindex(columns=['Estado','Año','Mes','Tasa de desempleo'])

In [23]:
df

,Estado,Año,Mes,Tasa de desempleo
0,Alaska,2010,Diciembre,8.0
1,Alaska,2010,Noviembre,8.1
2,Alaska,2010,Octubre,8.0
3,Alaska,2010,Septiembre,8.0
4,Alaska,2010,Agosto,7.9
...,...,...,...,...
115,Wyoming,2019,Mayo,3.4
116,Wyoming,2019,Abril,3.3
117,Wyoming,2019,Marzo,3.4
118,Wyoming,2019,Febrero,3.5


In [24]:
df.to_csv('Tasa_desempleo_API.csv', index=False)